In [0]:
dbutils.fs.ls('mnt/bronze/SalesLT')

[FileInfo(path='dbfs:/mnt/bronze/SalesLT/Address/', name='Address/', size=0, modificationTime=1747250257000),
 FileInfo(path='dbfs:/mnt/bronze/SalesLT/Customer/', name='Customer/', size=0, modificationTime=1747250244000),
 FileInfo(path='dbfs:/mnt/bronze/SalesLT/CustomerAddress/', name='CustomerAddress/', size=0, modificationTime=1747250221000),
 FileInfo(path='dbfs:/mnt/bronze/SalesLT/Product/', name='Product/', size=0, modificationTime=1747250229000),
 FileInfo(path='dbfs:/mnt/bronze/SalesLT/ProductCategory/', name='ProductCategory/', size=0, modificationTime=1747250251000),
 FileInfo(path='dbfs:/mnt/bronze/SalesLT/ProductDescription/', name='ProductDescription/', size=0, modificationTime=1747250225000),
 FileInfo(path='dbfs:/mnt/bronze/SalesLT/ProductModel/', name='ProductModel/', size=0, modificationTime=1747250221000),
 FileInfo(path='dbfs:/mnt/bronze/SalesLT/ProductModelProductDescription/', name='ProductModelProductDescription/', size=0, modificationTime=1747250225000),
 FileInf

In [0]:
dbutils.fs.ls('mnt/silver/')

[]

In [0]:
df=spark.read.format('parquet').load('/mnt/bronze/SalesLT/Address/Address.parquet')

In [0]:
from pyspark.sql.functions import from_utc_timestamp, date_format
from pyspark.sql.types import TimestampType

df=df.withColumn("ModifiedDate", date_format(from_utc_timestamp(df["ModifiedDate"].cast(TimestampType()),"UTC"),"yyyy-MM-dd"))

Data Transformation for Tables
****

In [0]:
table_name=[]
for i in dbutils.fs.ls('mnt/bronze/SalesLT/'):
  table_name.append(i.name.split('/')[0])
table_name  

['Address',
 'Customer',
 'CustomerAddress',
 'Product',
 'ProductCategory',
 'ProductDescription',
 'ProductModel',
 'ProductModelProductDescription',
 'SalesOrderDetail',
 'SalesOrderHeader']

In [0]:
from pyspark.sql.functions import from_utc_timestamp, date_format
from pyspark.sql.types import TimestampType

for i in table_name:
    path='/mnt/bronze/SalesLT/'+i+'/'+i+'.parquet'
    df=spark.read.format('parquet').load(path)
    column=df.columns

    for col in column:
        if "Date" in col or "date" in col:
            df=df.withColumn(col, date_format(from_utc_timestamp(df[col].cast(TimestampType()),"UTC"),"yyyy-MM-dd"))
    output_path='/mnt/silver/SalesLT/'+i+'/'
    df.write.mode('overwrite').format('delta').save(output_path)